In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [4]:
ori_data = pd.read_csv('HW4_creditcard.csv')

In [11]:
train_data,test_data,train_target,test_target = train_test_split(np.array(ori_data[ori_data.columns[:-1]]),np.array(ori_data[ori_data.columns[-1]]),test_size=0.3)

In [18]:
tree = DecisionTreeClassifier()
tree.fit(train_data,train_target)
score = tree.score(test_data, test_target)
tree
print(f'Accuracy : {score:.2f}')

Accuracy : 0.99
